In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)

/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym

2.0.0-alpha0
2.2.4-tf


/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWarning: Passing (type

In [2]:
import os
import csv
import time
import datetime
import random
import json
from collections import Counter
from math import sqrt
import gensim
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input,Conv2D,MaxPool2D,concatenate,Flatten,Dense,Dropout,Embedding,Reshape
from tensorflow.keras import Sequential,optimizers,losses
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from bs4 import BeautifulSoup
import logging
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import multiprocessing
import yaml
import jieba

In [3]:
class Config(object):
    #数据集路径
    dataSource = "../marriage.txt"
    stopWordSource = "../stopword.txt"
    #分词后保留大于等于最低词频的词
    miniFreq=0
    #统一输入文本序列的定长，取了所有序列长度的均值。超出将被截断，不足则补0
    sequenceLength = 20 
    batchSize=128
    epochs=20
    numClasses = 5
    #训练集的比例
    rate = 0.8  
    #生成嵌入词向量的维度
    embeddingSize = 200
    #卷积核数
    numFilters = 128
    #卷积核大小
    filterSizes = [1,2,3,4,5,6]
    dropoutKeepProb = 0.5
    #L2正则系数
    l2RegLambda = 0.01
# 实例化配置参数对象
config = Config()

In [4]:
config.batchSize

128

In [5]:
#中文语料
#设置输出日志
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

file = open("../marriage.txt") 
sentences=[]
with open('../marriage.txt') as fr:
    for line in fr.readlines():
        temp=line.strip().split('\t')
#         sentences.append(jieba.lcut(temp[0]))
        sentences.append(list(temp[0]))

model = word2vec.Word2Vec(sentences,size=config.embeddingSize,
                     min_count=config.miniFreq,
                     window=10,
                     workers=multiprocessing.cpu_count(),sg=1,
                     iter=20)
model.save('../word2VecModel')

In [6]:
model = gensim.models.Word2Vec.load('../word2VecModel')
model.wv.vocab.keys()

dict_keys(['你', '坤', '包', '好', '了', '啊', '说', '回', '信', '我', '现', '在', '发', '也', '不', '方', '便', '还', '会', '只', '是', '随', '注', '册', '一', '下', 'm', '特', '意', '去', '那', '个', '嘞', '登', '记', '过', '吗', '给', '就', '想', '珍', '爱', '网', '上', '面', '这', '样', '的', '谢', '像', '天', '有', '人', '联', '系', '用', '吧', '之', '前', '边', '跟', '嗯', '太', '改', '再', '班', '呢', '没', '大', '几', '年', '怎', '么', '办', '七', '二', '看', '反', '正', '然', '后', '知', '道', '载', '完', '高', '呃', '什', '证', '件', '但', '豪', '门', '哦', '行', '刚', '才', '点', '事', '先', '因', '为', '们', '暂', '时', '需', '要', '服', '务', '分', '钟', '菜', '家', '里', '外', '啥', '思', '呀', '早', '肯', '定', '久', '电', '话', '离', '婚', '法', '院', '判', '决', '书', '共', '取', '消', '确', '实', '忙', '等', '小', '孩', '弄', '手', '机', '告', '稿', '贾', '易', '玲', '喂', '理', '英', '文', '字', '麻', '烦', '拜', '升', '级', '选', '打', '钱', '工', '地', '八', '号', '咋', '复', '印', '出', '来', '头', '拿', '到', '三', '错', '干', '男', '学', '校', '稍', '微', '开', '对', '订', '都', '周', '六', '核', '他', '准', '备', '汤', '期', '四', '今', '候', '爸', '拌',

In [7]:
model['信']

/Library/Python/3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-1.62259936e-01, -1.01414181e-01, -9.01771933e-02,  1.38697177e-01,
       -2.15469211e-01,  1.72081217e-01,  8.95420536e-02,  8.58433470e-02,
        2.62835175e-01, -4.46709767e-02, -2.66334944e-04, -2.31834218e-01,
        1.01651862e-01, -1.06230192e-01,  1.49743512e-01, -2.03052536e-01,
        1.66971967e-01, -4.97035757e-02,  6.66131973e-02,  2.21493095e-01,
        9.60768759e-02, -3.17073101e-03, -1.34310842e-01,  2.14327380e-01,
        2.85391323e-02,  7.95748979e-02, -1.83805019e-01,  9.30562764e-02,
        1.01410877e-03, -1.53992116e-01,  2.69832730e-01,  1.91341475e-01,
        3.01492512e-01,  1.03915900e-01,  9.62394550e-02,  1.12453088e-01,
       -2.89852589e-01,  7.69588426e-02, -1.02878004e-01, -2.65263289e-01,
        5.85740097e-02, -4.51844901e-01,  5.51657341e-02,  1.36977777e-01,
       -1.17409185e-01,  1.56495303e-01, -3.65037084e-01,  1.15589097e-01,
       -3.55736136e-01, -1.35263637e-01, -1.09639261e-02,  5.38873255e-01,
        3.13623846e-01,  

In [8]:
# 数据预处理的类，生成训练集和测试集
class Dataset(object):
    def __init__(self, config):
        self.dataSource = config.dataSource
        self.stopWordSource = config.stopWordSource  
        # 每条输入的序列处理为定长
        self.sequenceLength = config.sequenceLength  
        self.embeddingSize = config.embeddingSize
        self.rate = config.rate
        self.miniFreq=config.miniFreq
        self.stopWordDict = {}
        self.trainReviews = []
        self.trainLabels = []
        self.evalReviews = []
        self.evalLabels = []
        self.wordEmbedding =None
        self.n_symbols=0
        self.wordToIndex = {}
        self.indexToWord = {}
        
    def readData(self, filePath):
        import random
        with open(filePath) as fr:
            data=fr.readlines()
        random.shuffle(data)
        with open(filePath,'w') as fw:
            fw.write(''.join(data))
        text=[]
        label=[]
        with open(filePath) as fr:
            for line in file:
                temp=line.strip().split('\t')
                text.append(temp[0])
                label.append(temp[1])
        print('data:',len(text),len(label))
#         texts = [jieba.cut(document) for document in text]
        texts = [list(document) for document in text]
        return texts, label
    
    def readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
    
    def getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        #中文
        model = gensim.models.Word2Vec.load('../word2VecModel')
        vocab = []
        wordEmbedding = []
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        wordEmbedding.append(np.zeros(self.embeddingSize))
        vocab.append("UNK")
        wordEmbedding.append(np.random.randn(self.embeddingSize))
        for word in words:
            vector =model[word]
            vocab.append(word)
            wordEmbedding.append(vector)           
        return vocab, np.array(wordEmbedding)
    
    def genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        allWords = [word for review in reviews for word in review]
        #去掉停用词
#         subWords = [word for word in allWords if word not in self.stopWordDict]
        #统计词频，排序
#         wordCount = Counter(subWords)
        wordCount = Counter(allWords)
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        #去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= self.miniFreq ]
        #获取词列表和顺序对应的预训练权重矩阵
        vocab, wordEmbedding = self.getWordEmbedding(words)
        
        self.wordEmbedding = wordEmbedding
        
        self.wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self.indexToWord = dict(zip(list(range(len(vocab))), vocab))
        self.n_symbols = len(self.wordToIndex) + 1
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self.wordToIndex, f)
        with open("../wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self.indexToWord, f)
            
    def reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论里面的词，根据词表，映射为index表示
        每条评论 用index组成的定长数组来表示
        """
        review=list(review)
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]
        return reviewVec

    def genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        reviews = []
        labels = []
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self.reviewProcess(x[i], self.sequenceLength, self.wordToIndex)
            reviews.append(reviewVec)
            labels.append([y[i]])    
        trainIndex = int(len(x) * rate)
        
        #trainReviews = sequence.pad_sequences(reviews[:trainIndex], maxlen=self.sequenceLength)
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        #evalReviews = sequence.pad_sequences(reviews[trainIndex:], maxlen=self.sequenceLength)
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")
        return trainReviews, trainLabels, evalReviews, evalLabels
         
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        #读取停用词
        self.readStopWord(self.stopWordSource)
        #读取数据集
        reviews, labels = self.readData(self.dataSource)
        #分词、去停用词
        #生成 词汇-索引 映射表和预训练权重矩阵，并保存
        self.genVocabulary(reviews)
        #初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self.genTrainEvalData(reviews, labels, self.rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels

data = Dataset(config)
data.dataGen()

data: 6321 6321


/Library/Python/3.7/site-packages/ipykernel_launcher.py:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [9]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (5056, 20)
train label shape: (5056, 1)
eval data shape: (1265, 20)


In [10]:
def convolution(config):
    sequence_length=config.sequenceLength
    embedding_dimension=config.embeddingSize
    
    inn = Input(shape=(sequence_length, embedding_dimension, 1))
    cnns = []
    filter_sizes=config.filterSizes
    for size in filter_sizes:
        conv = Conv2D(filters=config.numFilters, kernel_size=(size, embedding_dimension),
                            strides=1, padding='valid', activation='relu')(inn)
        pool = MaxPool2D(pool_size=(sequence_length-size+1, 1), padding='valid')(conv)
        cnns.append(pool)
    outt =concatenate(cnns)
    model = Model(inputs=inn, outputs=outt)
    return model

def cnn_mulfilter(n_symbols,embedding_weights,config):
    model =Sequential([
        Embedding(input_dim=n_symbols, output_dim=config.embeddingSize,
                        weights=[embedding_weights],
                        input_length=config.sequenceLength, trainable=True),
        Reshape((config.sequenceLength, config.embeddingSize, 1)),
        convolution(config),
        Flatten(),
#         Dense(10, activation='relu',kernel_regularizer=regularizers.l2(config.l2RegLambda)),
#         Dropout(config.dropoutKeepProb),
        Dense(6, activation='softmax')])
    
#     model.compile(loss='SparseCategoricalCrossentropy',optimizer='adam',metrics=['accuracy'])
    model.compile(optimizer=optimizers.Adam(lr=1e-3),
                 loss=losses.SparseCategoricalCrossentropy(),
                 metrics=['accuracy'])
    return model
    
wordEmbedding = data.wordEmbedding
n_symbols=data.n_symbols
model = cnn_mulfilter(n_symbols,wordEmbedding,config)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 200)           236200    
_________________________________________________________________
reshape (Reshape)            (None, 20, 200, 1)        0         
_________________________________________________________________
model (Model)                (None, 1, 1, 768)         538368    
_________________________________________________________________
flatten (Flatten)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 4614      
Total params: 779,182
Trainable params: 779,182
Non-trainable params: 0
_________________________________________________________________


In [11]:
x_train = data.trainReviews
y_train = data.trainLabels
x_eval = data.evalReviews
y_eval = data.evalLabels

# y_train = to_categorical(y_train)
# y_eval = to_categorical(y_eval)

wordEmbedding = data.wordEmbedding
n_symbols=data.n_symbols

reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=10, mode='auto')

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('./model/best_model/model_{epoch:02d}-{val_accuracy:.2f}.hdf5', save_best_only=True, save_weights_only=True)
# history = model.fit(x_train, y_train, batch_size=config.batchSize, epochs=config.epochs, validation_split=0.2,shuffle=True, callbacks=[reduce_lr,early_stopping,model_checkpoint])
history = model.fit(x_train, y_train, batch_size=config.batchSize, epochs=config.epochs, validation_split=0.2,shuffle=True, callbacks=[model_checkpoint])

#验证
scores = model.evaluate(x_eval, y_eval)

#保存模型
yaml_string = model.to_yaml()
with open('./model/textCNN.yml', 'w') as outfile:
    outfile.write( yaml.dump(yaml_string, default_flow_style=True) )
model.save_weights('./model/textCNN.h5')

print('test_loss: %f, accuracy: %f' % (scores[0], scores[1]))

Train on 4044 samples, validate on 1012 samples
Epoch 1/20
4044/4044 [==============================] - 5s 1ms/sample - loss: 1.1108 - accuracy: 0.5678 - val_loss: 0.5969 - val_accuracy: 0.7777
Epoch 2/20
4044/4044 [==============================] - 4s 901us/sample - loss: 0.5054 - accuracy: 0.8200 - val_loss: 0.4123 - val_accuracy: 0.8587
Epoch 3/20
4044/4044 [==============================] - 3s 863us/sample - loss: 0.3492 - accuracy: 0.8848 - val_loss: 0.3780 - val_accuracy: 0.8715
Epoch 4/20
4044/4044 [==============================] - 4s 940us/sample - loss: 0.2635 - accuracy: 0.9169 - val_loss: 0.3761 - val_accuracy: 0.8696
Epoch 5/20
4044/4044 [==============================] - 4s 919us/sample - loss: 0.2065 - accuracy: 0.9325 - val_loss: 0.3795 - val_accuracy: 0.8706
Epoch 6/20
4044/4044 [==============================] - 4s 941us/sample - loss: 0.1625 - accuracy: 0.9530 - val_loss: 0.3863 - val_accuracy: 0.8735
Epoch 7/20
4044/4044 [==============================] - 4s 914us/s

In [12]:
dir(losses)

['BinaryCrossentropy',
 'CategoricalCrossentropy',
 'CategoricalHinge',
 'CosineSimilarity',
 'Hinge',
 'Huber',
 'KLD',
 'KLDivergence',
 'LogCosh',
 'LogLoss',
 'Loss',
 'MAE',
 'MAPE',
 'MSE',
 'MSLE',
 'MeanAbsoluteError',
 'MeanAbsolutePercentageError',
 'MeanSquaredError',
 'MeanSquaredLogarithmicError',
 'Poisson',
 'Reduction',
 'SparseCategoricalCrossentropy',
 'SquaredHinge',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'binary_crossentropy',
 'categorical_crossentropy',
 'categorical_hinge',
 'cosine_similarity',
 'deserialize',
 'get',
 'hinge',
 'kld',
 'kullback_leibler_divergence',
 'logcosh',
 'mae',
 'mape',
 'mean_absolute_error',
 'mean_absolute_percentage_error',
 'mean_squared_error',
 'mean_squared_logarithmic_error',
 'mse',
 'msle',
 'poisson',
 'serialize',
 'sparse_categorical_crossentropy',
 'squared_hinge']

In [13]:
y_train

array([[2.],
       [0.],
       [1.],
       ...,
       [4.],
       [3.],
       [0.]], dtype=float32)

In [14]:
to_categorical(y_train)

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [15]:
n_symbols

1181